In [1]:
!pip install -U pinecone google-generativeai PyMuPDF pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.9 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [3]:
!pip install -q tqdm

In [4]:
from pinecone import Pinecone
from getpass import getpass
from uuid import uuid4

# Step 1: API key
PINECONE_API_KEY = getpass("🔐 Pinecone API Key: ")

# Step 2: Connect
pc = Pinecone(api_key=PINECONE_API_KEY)

# Step 3: Create index for server-side embedding
index_name = "sherlock"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"multilingual-e5-large",
            "field_map":{"text": "chunk_text"}
        }
    )



🔐 Pinecone API Key: ··········


In [ ]:
import os
from google.colab import files


if not os.path.exists("docs"):
    os.makedirs("docs")


uploaded = files.upload()


for filename in uploaded.keys():
    os.rename(filename, f"docs/{filename}")


Saving A Study in Scarlet.pdf to A Study in Scarlet.pdf


In [ ]:
import os
from uuid import uuid4
from getpass import getpass
from pinecone import Pinecone
import PyPDF2
import google.generativeai as genai

def extract_chunks_from_pdf(file_path):
    chunks = []
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()
            if text:
                text = text.replace("\n", "\n ")
                for para in text.split("\n\n"):
                    clean_para = para.strip()
                    if len(clean_para) > 100:
                        chunks.append(f"[Page {page_num+1}]\n{clean_para}")
    return chunks


In [ ]:
records = []
for filename in os.listdir("docs"):
    filepath = os.path.join("docs", filename)
    chunks = extract_chunks_from_pdf(filepath)
    for chunk in chunks:
        records.append({
            "_id": str(uuid4()),
            "chunk_text": chunk
        })

In [ ]:
with open("records1.txt", "w") as f:
    for item in records:
        f.write(f"{item}\n")


In [8]:
# Target the index
dense_index = pc.Index(index_name)

In [ ]:
import time
# Upsert the records into a namespace

def batched_upsert(index, records, namespace=None, batch_size=90):
    from itertools import islice

    def chunks(iterable, size):
        it = iter(iterable)
        while chunk := list(islice(it, size)):
            yield chunk

    for i, batch in enumerate(chunks(records, batch_size), start=1):
        index.upsert_records("a-study-in-scarlet", batch)

        if i % 3 == 0:
          print(f"⏸️  Sleeping 61 seconds after batch {i} to respect token limit...")
          time.sleep(61)

# 🚀 Upsert in batches with namespace
batched_upsert(index=dense_index, records=records, namespace="ns1")


print(f"✅ Upserted {len(records)} records into Pinecone index '{index_name}'")

✅ Upserted 55 records into Pinecone index 'sherlock'


In [ ]:
dense_index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'a-study-in-scarlet': {'vector_count': 55},
                'ns1': {'vector_count': 806}},
 'total_vector_count': 861,
 'vector_type': 'dense'}

In [5]:
import os
import time
from getpass import getpass
import google.generativeai as genai
GEMMA_API_KEY = getpass("🔐 Gemma API Key: ")

genai.configure(api_key=GEMMA_API_KEY)

model = genai.GenerativeModel("gemma-3-12b-it")



🔐 Gemma API Key: ··········


In [11]:
# Helper to extract text from search results

def extract_context_from_results(results):
    hits = results.get('result', {}).get('hits', [])
    if not hits:
        return "No context found."
    return "\n\n".join(hit.get('fields', {}).get('chunk_text', '') for hit in hits)


In [19]:

#Query
while True:
    query = input("\n Ask a question (or type 'q' to quit): ").strip()
    if query.lower() == "q":
        break

    results = dense_index.search(
        namespace="ns1",
        query={
            "inputs": {"text": query},
            "top_k": 8
        },
        fields=["chunk_text"]
    )

    context = extract_context_from_results(results)
    prompt = f"""Answer the following question using the provided context below. If the answer is not in the context, say so.

### Question:
{query}

### Context:
{context}
"""

    response = model.generate_content(prompt)
    print("\n💡 Answer:")
    print(response.text)


 Ask a question (or type 'q' to quit): who is sherlock holmes's brother

💡 Answer:
According to the provided text, Sherlock Holmes's brother is Mycroft. The text states, "It was after tea on a summer evening...he began to talk to me about his brother." Later, it clarifies, "Because my brother Mycroft possesses it [observation and deduction] in a larger degree than I do."





 Ask a question (or type 'q' to quit): what are the traits of sherlock holmes? 

💡 Answer:
Based on the provided text, here are the traits of Sherlock Holmes:

*   **Highly intelligent:** He possesses a pre-eminent intelligence and is a skilled reasoner.
*   **Observant:** He has an extraordinary faculty for observation.
*   **Logical:** He is a logician and values seeing things "exactly as they are."
*   **Deductive:** He has a peculiar facility for deduction.
*   **Unemotional/Detached:** He is described as having an "unemotional character" and a "brain without a heart." He avoids strong emotions, viewing them 